Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-with-automated-machine-learning-step.png)

# Azure Machine Learning Pipeline with AutoMLStep (Udacity Course 2)
This notebook demonstrates the use of AutoMLStep in Azure Machine Learning Pipeline.

## Introduction
In this example we showcase how you can use AzureML Dataset to load data for AutoML via AML Pipeline. 

If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you have executed the [configuration](https://aka.ms/pl-config) before running this notebook.

In this notebook you will learn how to:
1. Create an `Experiment` in an existing `Workspace`.
2. Create or Attach existing AmlCompute to a workspace.
3. Define data loading in a `TabularDataset`.
4. Configure AutoML using `AutoMLConfig`.
5. Use AutoMLStep
6. Train the model using AmlCompute
7. Explore the results.
8. Test the best fitted model.

## Azure Machine Learning and Pipeline SDK-specific imports

In [42]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.56.0


## Initialize Workspace
Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json

In [43]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-261984
aml-quickstarts-261984
southcentralus
9b72f9e6-56c5-4c16-991b-19c652994860


## Create an Azure ML experiment
Let's create an experiment named "automlstep-classification" and a folder to hold the training scripts. The script runs will be recorded under the experiment in Azure.

The best practice is to use separate folders for scripts and its dependent files for each step and specify that folder as the `source_directory` for the step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the `source_directory` would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the `source_directory` of the step.

*Udacity Note:* There is no need to create an Azure ML experiment, this needs to re-use the experiment that was already created


In [ ]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'bank-automl-job-exp-1'
project_folder = './pipeline-project'

experiment = Experiment(ws, experiment_name)
experiment

### Create or Attach an AmlCompute cluster
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for your AutoML run. In this tutorial, you get the default `AmlCompute` as your training compute resource.

**Udacity Note** There is no need to create a new compute target, it can re-use the previous cluster

In [44]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Data

**Udacity note:** Make sure the `key` is the same name as the dataset that is uploaded, and that the description matches. If it is hard to find or unknown, loop over the `ws.datasets.keys()` and `print()` them.
If it *isn't* found because it was deleted, it can be recreated with the link that has the CSV 

In [45]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "BankMarketing Dataset"
description_text = "Bank Marketing DataSet for Udacity Course 2"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,32950.00,32950.00,32950.00,32950.00,32950.00,32950.00,32950.00,32950.00,32950.00,32950.00
mean,40.04,257.34,2.56,962.17,0.17,0.08,93.57,-40.52,3.62,5166.86
std,10.43,257.33,2.76,187.65,0.50,1.57,0.58,4.62,1.74,72.21
min,17.00,0.00,1.00,0.00,0.00,-3.40,92.20,-50.80,0.63,4963.60
25%,32.00,102.00,1.00,999.00,0.00,-1.80,93.08,-42.70,1.34,5099.10
50%,38.00,179.00,2.00,999.00,0.00,1.10,93.75,-41.80,4.86,5191.00
75%,47.00,318.00,3.00,999.00,0.00,1.40,93.99,-36.40,4.96,5228.10
max,98.00,4918.00,56.00,999.00,7.00,1.40,94.77,-26.90,5.04,5228.10


### Review the Dataset Result

You can peek the result of a TabularDataset at any range using `skip(i)` and `take(j).to_pandas_dataframe()`. Doing so evaluates only `j` records for all the steps in the TabularDataset, which makes it fast even against large datasets.

`TabularDataset` objects are composed of a list of transformation steps (optional).

In [46]:
dataset.take(5).to_pandas_dataframe()

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.80,92.89,-46.20,1.30,5099.10,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.10,93.99,-36.40,4.86,5191.00,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.80,92.89,-46.20,1.31,5099.10,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.40,94.47,-41.80,4.97,5228.10,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.40,93.92,-42.70,4.96,5228.10,no


## Train
This creates a general AutoML settings object.
**Udacity notes:** These inputs must match what was used when training in the portal. `label_column_name` has to be `y` for example.

In [47]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="y",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

#### Create Pipeline and AutoMLStep

You can define outputs for the AutoMLStep using TrainingOutput.

In [37]:
from azureml.pipeline.core import PipelineData, TrainingOutput

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

Create an AutoMLStep.

In [48]:
automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)

In [49]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])

In [40]:
pipeline_run = experiment.submit(pipeline)

Submitted PipelineRun ec22d618-510f-49bb-ac75-79979c4156d2
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/ec22d618-510f-49bb-ac75-79979c4156d2?wsid=/subscriptions/9b72f9e6-56c5-4c16-991b-19c652994860/resourcegroups/aml-quickstarts-261984/workspaces/quick-starts-ws-261984&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254


In [39]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

ERROR:azureml.data._dataset_client:[NOT_SUPPORTED_API_USE_ATTEMPT] The [_DatasetClient.get] API has been deprecated and is no longer supported
ERROR:azureml.data._dataset_client:[NOT_SUPPORTED_API_USE_ATTEMPT] The [_DatasetClient.get] API has been deprecated and is no longer supported
ERROR:azureml.data._dataset_client:[NOT_SUPPORTED_API_USE_ATTEMPT] The [_DatasetClient.get] API has been deprecated and is no longer supported
ERROR:azureml.data._dataset_client:[NOT_SUPPORTED_API_USE_ATTEMPT] The [_DatasetClient.get] API has been deprecated and is no longer supported


In [41]:
pipeline_run.wait_for_completion()

PipelineRunId: ec22d618-510f-49bb-ac75-79979c4156d2
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/ec22d618-510f-49bb-ac75-79979c4156d2?wsid=/subscriptions/9b72f9e6-56c5-4c16-991b-19c652994860/resourcegroups/aml-quickstarts-261984/workspaces/quick-starts-ws-261984&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

PipelineRun Execution Summary
PipelineRun Status: Finished
{'runId': 'ec22d618-510f-49bb-ac75-79979c4156d2', 'status': 'Completed', 'startTimeUtc': '2024-07-08T16:30:27.757808Z', 'endTimeUtc': '2024-07-08T16:30:29.403769Z', 'services': {}, 'properties': {'azureml.runsource': 'azureml.PipelineRun', 'runSource': 'SDK', 'runType': 'SDK', 'azureml.parameters': '{}', 'azureml.continue_on_step_failure': 'False', 'azureml.continue_on_failed_optional_input': 'True', 'azureml.pipelineComponent': 'pipelinerun', 'azureml.pipelines.stages': '{"Initialization":null,"Execution":{"StartTime":"2024-07-08T16:30:27.9698526+00:00","EndTime":"2024-07-08T16:30:29.2719099+00:00","Status":

'Finished'

## Examine Results

### Retrieve the metrics of all child runs
Outputs of above run can be used as inputs of other steps in pipeline. In this tutorial, we will examine the outputs by retrieve output data and running some tests.

In [36]:
metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

In [35]:
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

,d8b0915e-5d69-4b8d-9d02-fd5c59f8e445_8,d8b0915e-5d69-4b8d-9d02-fd5c59f8e445_4,d8b0915e-5d69-4b8d-9d02-fd5c59f8e445_6,d8b0915e-5d69-4b8d-9d02-fd5c59f8e445_11,d8b0915e-5d69-4b8d-9d02-fd5c59f8e445_14,d8b0915e-5d69-4b8d-9d02-fd5c59f8e445_31,d8b0915e-5d69-4b8d-9d02-fd5c59f8e445_17,d8b0915e-5d69-4b8d-9d02-fd5c59f8e445_26,d8b0915e-5d69-4b8d-9d02-fd5c59f8e445_35,d8b0915e-5d69-4b8d-9d02-fd5c59f8e445_33,...,d8b0915e-5d69-4b8d-9d02-fd5c59f8e445_43,d8b0915e-5d69-4b8d-9d02-fd5c59f8e445_46,d8b0915e-5d69-4b8d-9d02-fd5c59f8e445_29,d8b0915e-5d69-4b8d-9d02-fd5c59f8e445_41,d8b0915e-5d69-4b8d-9d02-fd5c59f8e445_45,d8b0915e-5d69-4b8d-9d02-fd5c59f8e445_40,d8b0915e-5d69-4b8d-9d02-fd5c59f8e445_42,d8b0915e-5d69-4b8d-9d02-fd5c59f8e445_51,d8b0915e-5d69-4b8d-9d02-fd5c59f8e445_44,d8b0915e-5d69-4b8d-9d02-fd5c59f8e445_52
f1_score_macro,[0.7472056145957435],[0.6882189452748133],[0.6861070168635817],[0.6211041442218217],[0.728625486922649],[0.7449910772987696],[0.6883070554741297],[0.729809956769593],[0.7535318458192026],[0.7383120665007677],...,[0.7581054205208292],[0.7587603517261192],[0.5435442173552509],[0.7702305640099276],[0.4703423886834914],[0.7553038539457346],[0.7675783868585824],[0.7620646638270467],[0.7624430143718348],[0.7516168316494881]
f1_score_weighted,[0.9048597237768088],[0.8909573471197098],[0.828268119025813],[0.7947572604737844],[0.9008709204263049],[0.9023446319059719],[0.8321144764831118],[0.900539981658476],[0.9052670264610582],[0.9040394943716457],...,[0.9051449252597966],[0.9060267190237284],[0.8531514455159263],[0.9096831483089586],[0.8353395018439429],[0.9057752295479862],[0.9109396766887173],[0.9096674670075894],[0.9084123983203316],[0.9083918451311422]
norm_macro_recall,[0.44210489268255637],[0.2915131509483242],[0.7100789668183762],[0.48295165111596416],[0.38492480276819174],[0.45389804889163043],[0.6949163374067098],[0.3930308031720098],[0.47321000209318553],[0.40423675596974684],...,[0.501605084403544],[0.49620911109999666],[0.07583259701359824],[0.5277041458042742],[0.0],[0.4782882927940695],[0.4962813537909816],[0.4769934814864212],[0.49286371879439916],[0.4316787904721153]
matthews_correlation,[0.5016093372462171],[0.42149687086703563],[0.4866303814711651],[0.3331696738020599],[0.47541827231149086],[0.4931567582740051],[0.4812581984247661],[0.47422181829067633],[0.5096832200958874],[0.4933932803541404],...,[0.5166443905575122],[0.5184688170203535],[0.20382267725353595],[0.5407689726067446],[0.0],[0.512951950649998],[0.5384170670164895],[0.5292900090108754],[0.52709053271067],[0.5179225130984195]
accuracy,[0.9104704097116844],[0.9071320182094081],[0.7930197268588771],[0.7532625189681336],[0.9101669195751139],[0.9062215477996965],[0.7984825493171472],[0.9089529590288316],[0.908649468892261],[0.9125948406676783],...,[0.906525037936267],[0.9080424886191198],[0.8922610015174507],[0.910773899848255],[0.8880121396054628],[0.9089529590288316],[0.9144157814871017],[0.9141122913505311],[0.9113808801213961],[0.9159332321699545]
f1_score_micro,[0.9104704097116844],[0.9071320182094081],[0.7930197268588771],[0.7532625189681336],[0.9101669195751139],[0.9062215477996965],[0.7984825493171472],[0.9089529590288316],[0.908649468892261],[0.9125948406676783],...,[0.9065250379362669],[0.90804248861912],[0.8922610015174507],[0.9107738998482551],[0.8880121396054628],[0.9089529590288316],[0.9144157814871017],[0.9141122913505311],[0.9113808801213961],[0.9159332321699545]
log_loss,[0.19932095640960198],[0.22090578349014284],[0.38891248994946426],[0.5545149044260101],[0.2070233337832907],[0.19288051467184195],[0.41939630393798843],[0.19873978109892296],[0.18634083858083422],[0.190678128934451],...,[0.2152399193399559],[0.1991372831387409],[0.2329764408244545],[0.1967811528467674],[0.35065199384648504],[0.5681612043391066],[0.18014255794088174],[0.18995872559499605],[0.18071078415662217],[0.1950773884444693]
recall_score_micro,[0.9104704097116844],[0.9071320182094081],[0.7930197268588771],[0.7532625189681336],[0.9101669195751139],[0.9062215477996965],[

### Retrieve the Best Model

In [29]:
# Retrieve best model from Pipeline Run
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

In [50]:
import pickle

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, working_dir='/mnt/batch/tasks/shared/LS_root/mounts/clusters/pipeline-compute/code/Users/odl_user_261984')),
                                                 ('prefittedsoftvotingclassifier',
                                                  PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), e...666666666667, reg_lambda=1.9791666666666667, subsample=1, tree_method='hist'))]))], flatten_transform=False, weights=[0.16666666666666666, 0.08333333333333333, 0.25, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333]))],
                                       'verbose': False},
                             y_transformer={},
                             y_transformer_name='LabelEncoder')

In [34]:
best_model.steps

[('datatransformer',
  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=False, is_onnx_compatible=False, task='classification')),
 ('prefittedsoftvotingclassifier',
  PreFittedSoftVotingClassifier(classification_labels=numpy.array([0, 1]), estimators=[('32', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=0.7, eta=0.4, gamma=5, max_depth=6, max_leaves=0, n_estimators=100, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, reg_alpha=1.7708333333333335, reg_lambda=1.5625, subsample=0.5, tree_method='auto'))], verbose=False)), ('0', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('lightgbmclas

### Test the Model
#### Load Test Data
For the test data, it should have the same preparation step as the train data. Otherwise it might get failed at the preprocessing step.

In [33]:
dataset_test = Dataset.Tabular.from_delimited_files(path='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv')
df_test = dataset_test.to_pandas_dataframe()
df_test = df_test[pd.notnull(df_test['y'])]

y_test = df_test['y']
X_test = df_test.drop(['y'], axis=1)

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


#### Testing Our Best Fitted Model

We will use confusion matrix to see how our model works.

In [32]:
from sklearn.metrics import confusion_matrix
ypred = best_model.predict(X_test)
cm = confusion_matrix(y_test, ypred)

In [ ]:
# Visualize the confusion matrix
pd.DataFrame(cm).style.background_gradient(cmap='Blues', low=0, high=0.9)

## Publish and run from REST endpoint

Run the following code to publish the pipeline to your workspace. In your workspace in the portal, you can see metadata for the pipeline including run history and durations. You can also run the pipeline manually from the portal.

Additionally, publishing the pipeline enables a REST endpoint to rerun the pipeline from any HTTP library on any platform.


In [51]:
published_pipeline = pipeline_run.publish_pipeline(
    name="Bankmarketing Train", description="Training bankmarketing pipeline", version="1.0")

published_pipeline


Name,Id,Status,Endpoint
Bankmarketing Train,252e6161-ab29-4b3d-8994-0e914c9e7075,Active,REST Endpoint


Authenticate once again, to retrieve the `auth_header` so that the endpoint can be used

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()



Get the REST url from the endpoint property of the published pipeline object. You can also find the REST url in your workspace in the portal. Build an HTTP POST request to the endpoint, specifying your authentication header. Additionally, add a JSON payload object with the experiment name and the batch size parameter. As a reminder, the process_count_per_node is passed through to ParallelRunStep because you defined it is defined as a PipelineParameter object in the step configuration.

Make the request to trigger the run. Access the Id key from the response dict to get the value of the run id.


In [52]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "pipeline-rest-endpoint"}
                        )

In [53]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

Submitted pipeline run:  d5fcac23-4718-476e-bb2a-e97facd35337


Use the run id to monitor the status of the new run. This will take another 10-15 min to run and will look similar to the previous pipeline run, so if you don't need to see another pipeline run, you can skip watching the full output.

In [54]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments["pipeline-rest-endpoint"], run_id)
RunDetails(published_pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

ERROR:azureml.data._dataset_client:[NOT_SUPPORTED_API_USE_ATTEMPT] The [_DatasetClient.get] API has been deprecated and is no longer supported
